## Problem Description

The city is divided into several districts, each with a waste collection point. The waste management company has a fleet of garbage trucks, all starting from the same depot. Each truck has a maximum capacity, and each collection point generates a certain amount of waste daily. The distances between all points (depot and collection points) are known. The objective is to find the optimal routes for the trucks to collect all waste, minimizing the total distance traveled, and hence, fuel consumption and CO2 emissions.

### Mathematical Model

This problem can be modeled as a Capacitated Vehicle Routing Problem (CVRP). The main components of the model are:

**Variables**: Binary variables x[i,j,k] (1 if truck k travels from point i to point j, 0 otherwise) and continuous variables y[i,k] representing the load of truck k after visiting point i.

**Objective function**: Minimize the total distance traveled by all trucks, which is the sum of the distances between points i and j for all trucks k if they travel that route.

**Constraints**: Each collection point is visited exactly once by exactly one truck. Each truck starts and ends its route at the depot. The total waste collected by each truck does not exceed its capacity. Subtour elimination constraints to prevent trucks from making unnecessary detours.

In [ ]:
!pip3 install ortools

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536],
        [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084],
        [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400],
        [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232],
        [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118],
        [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582],
        [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354],
        [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730],
        [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 616],
        [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342],
        [536, 1084, 400, 1232, 1118, 582, 354, 730, 616, 342, 0]
    ]
    data['demands'] = [0, 18, 26, 31, 42, 15, 21, 12, 16, 29, 23]
    data['vehicle_capacities'] = [100, 100, 100, 100]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            next_node_index = manager.IndexToNode(solution.Value(routing.NextVar(index)))
            route_distance += routing.GetArcCostForVehicle(node_index, next_node_index, vehicle_id)
            plan_output += f' {node_index} -> '
            index = solution.Value(routing.NextVar(index))
        node_index = manager.IndexToNode(index)
        plan_output += f'{node_index}\n'
        plan_output += f'Distance of the route: {route_distance}m\n'
        print(plan_output)


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        return data['distance_matrix'][manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        return data['demands'][manager.IndexToNode(from_index)]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 ->  9 ->  10 ->  2 ->  6 -> 0
Distance of the route: 1712m

Route for vehicle 1:
 0 ->  1 ->  3 ->  4 -> 0
Distance of the route: 1552m

Route for vehicle 2:
 0 ->  8 ->  5 ->  7 -> 0
Distance of the route: 1004m

Route for vehicle 3:
 0 -> 0
Distance of the route: 0m

